In [ ]:
import warnings
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
warnings.simplefilter('ignore', category=UserWarning)
warnings.filterwarnings('ignore',category=ImportWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

from scipy.stats import randint, loguniform
from imblearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from category_encoders import LeaveOneOutEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.feature_selection import RFE
from imblearn.over_sampling import ADASYN
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler, PowerTransformer, QuantileTransformer
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split, HalvingRandomSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier

In [ ]:
# List of continuous features to be binned into intervals
ordinal_features = ['week','remaining_downs']
binned_features = ['play_sequence_game','game_seconds_remaining']

# Train test split parameters
test_holdout_percentage = 0.10

# RFE feature selection
rfc_estimators = 50
rfc_max_features = 'sqrt'

# Define list of scalers being used
scaler_list = [StandardScaler(), 
               RobustScaler(), 
               MinMaxScaler(), 
               MaxAbsScaler(), 
               PowerTransformer(),
               QuantileTransformer(output_distribution='normal'), 
               QuantileTransformer(output_distribution='uniform'),
               ]

# Specify the HalvingRandomSearchCV parameters
scoring = 'accuracy'                        
n_cross_validation = 3
halving_parameter = 5.0
max_resource = 1000
resource_divisor = 5.0
min_resource = int(round((max_resource / resource_divisor), 0))

In [ ]:
# Create timer to calculate total workbook time in hours
start_time = time.time()

## IMPORT PROCESSED NFL-DATA-PY CSV FILE
##### https://pypi.org/project/nfl-data-py/

In [ ]:
# Import csv file from nfl-data-py
df = pd.read_csv(r'/Users/ttas2/Documents/Python/nfl-machine-learning-models/output_files/nfl_post_processing_run_pass_classification_data.csv')

df.shape

In [ ]:
# Convert binary columns to integers
binary_columns = df.columns[df.isin([0,1]).all()].tolist()
df[binary_columns] = df[binary_columns].apply(pd.to_numeric, downcast='integer', errors='coerce', axis=1)

In [ ]:
# Print columns with missing values
print(df.columns[df.isnull().any()].tolist())

In [ ]:
# Target frequency
target_count = df.play_type.value_counts(normalize=True)

target_count

## TRAIN TEST SPLIT
##### https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
# split data into target and feature datasets
X, y = df.loc[:, df.columns != 'play_type'], df['play_type']

initial_features = X.columns.to_list()

# Create train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_holdout_percentage, random_state=67)


## BASELINE MODEL
##### https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html

In [ ]:
# Create and fit baseline model to compare performance
baseline_model = DummyClassifier(strategy='most_frequent', random_state=67)
baseline_model.fit(X_train, y_train)

# Calculate model accuracy on test data
y_baseline_pred = baseline_model.predict(X_test)

print(f"Baseline accuracy: {round(accuracy_score(y_test,y_baseline_pred)*100, 1)}%")


## MODEL PIPELINE
##### https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html
##### https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html
##### https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html
##### https://www.statsmodels.org/dev/generated/statsmodels.stats.outliers_influence.variance_inflation_factor.html
##### https://imbalanced-learn.org/stable/references/over_sampling.html
##### https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html
##### https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
##### https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
##### https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html
##### https://xgboost.readthedocs.io/en/stable/parameter.html

In [ ]:
# Create feature type lists for column transform stage of the pipeline
categorical_features = list(X_train.select_dtypes(include='object'))
boolean_features = X_train.columns[X_train.isin([0, 1]).all()].tolist()
numeric_features = list(set(X_train.columns) - set(ordinal_features) - set(categorical_features) - set(boolean_features) - set(binned_features))

print('Boolean features:', len(boolean_features)) #, ':', boolean_features)
print('Numeric features:', len(numeric_features), ':', numeric_features)
print('Ordinal features:', len(ordinal_features), ':', ordinal_features)
print('Binned features:', len(binned_features), ':', binned_features)
print('Categorical features:', len(categorical_features), ':', categorical_features)
print(' ')
print(len(initial_features), 'total features')

In [ ]:
# Custom transformer for IQR outlier exclusion
class IQRTransformer:
    def __init__(self, numerical_cols):
        self.numerical_cols = numerical_cols
        self.lower_bound = None
        self.upper_bound = None

    def fit(self, x, y=None):
        if isinstance(x, pd.DataFrame):
            # Calculate the IQR for each numerical column
            q1 = x[self.numerical_cols].quantile(0.25)
            q3 = x[self.numerical_cols].quantile(0.75)
            iqr = q3 - q1

            # Define the lower and upper bounds for outliers
            self.lower_bound = (q1 - 1.5 * iqr).to_dict()
            self.upper_bound = (q3 + 1.5 * iqr).to_dict()
        else:
            # Calculate the IQR for each numerical column
            q1 = np.quantile(x[:, :], 0.25, axis=0)
            q3 = np.quantile(x[:, :], 0.75, axis=0)
            iqr = q3 - q1

            # Define the lower and upper bounds for outliers
            self.lower_bound = (q1 - 1.5 * iqr).tolist()
            self.upper_bound = (q3 + 1.5 * iqr).tolist()

        return self

    def transform(self, x, y=None):
        if isinstance(x, pd.DataFrame):
            # Exclude outliers based on the IQR for each numerical column
            x_outlier_removed = x.copy()
            for col in self.numerical_cols:
                if col in self.lower_bound and col in self.upper_bound:
                    x_outlier_removed = x_outlier_removed[
                        (x_outlier_removed[col] >= self.lower_bound[col]) & (x_outlier_removed[col] <= self.upper_bound[col])
                    ].dropna()
        else:
            # Exclude outliers based on the IQR for each numerical column
            x_outlier_removed = x.copy()
            for i, col in enumerate(self.numerical_cols):
                if col in self.lower_bound and col in self.upper_bound:
                    lower_bound = self.lower_bound[col]
                    upper_bound = self.upper_bound[col]
                    x_outlier_removed = x_outlier_removed[
                        (x_outlier_removed[:, i] >= lower_bound) & (x_outlier_removed[:, i] <= upper_bound)
                    ].dropna()

        return x_outlier_removed

In [ ]:
# Create custom transformer for selecting a feature scaler: StandardScaler, RobustScaler, MinMaxScaler, MaxAbsScaler
class ScalerSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, scaler=StandardScaler()):
        super().__init__()
        self.scaler = scaler

    def fit(self, x, y=None):
        return self.scaler.fit(x)

    def transform(self, x, y=None):
        return self.scaler.transform(x)
    
# scaler and encoder options
feature_scaler = ScalerSelector()

In [ ]:
# Specify the transformations per data type
num_trans = Pipeline(steps=[('num_imputer', SimpleImputer(missing_values=np.nan, strategy='median')),
                            ('num_outlier', IQRTransformer(numerical_cols=numeric_features)),
                            ('num_scaler', feature_scaler),
                           ])

cat_trans = Pipeline(steps=[('cat_imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
                            ('cat_encoder', LeaveOneOutEncoder(handle_missing='value', handle_unknown='value', random_state=67)),
                            ])

ord_trans = Pipeline(steps=[('ord_imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
                            ('ord_encoder', OrdinalEncoder(categories='auto', handle_unknown='use_encoded_value', unknown_value=-1)),
                           ])

bin_trans = Pipeline(steps=[('bin_imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
                            ('bin_discretizer', KBinsDiscretizer(subsample=None, random_state=67)),
                            ('bin_scaler', feature_scaler),
                           ])

preprocessing = ColumnTransformer(transformers=[('num_transform', num_trans, numeric_features),
                                                ('cat_transform', cat_trans, categorical_features),
                                                ('ord_transform', ord_trans, ordinal_features),
                                                ('bin_transform', bin_trans, binned_features),
                                                ],
                                  remainder='passthrough',
                                  )

In [ ]:
# Define the models
models = [
    ('RandomForest', RandomForestClassifier()),
    ('ExtraTrees', ExtraTreesClassifier()),
    ('GradientBoosting', GradientBoostingClassifier()),
    ('AdaBoost', AdaBoostClassifier()),
    ('XGBoost', XGBClassifier()),
]

# Create and run the pipeline
for model_name, model in models:
    pipeline = Pipeline([
        ('pre', preprocessing),
        ('sel', RFE(RandomForestClassifier(n_estimators=rfc_estimators, max_features=rfc_max_features, class_weight='balanced_subsample', n_jobs=6, random_state=67))),
        ('ovr', ADASYN(sampling_strategy='not majority', random_state=67)),
        ('clf', model)
    ])
 
    params = {}

    if model_name == 'RandomForest':
        params = {
            'pre__cat_transform__cat_encoder__sigma': loguniform(0.001, 0.05),
            'pre__num_transform__num_scaler': scaler_list,
            'pre__bin_transform__bin_discretizer__n_bins': randint(10, 25),
            'pre__bin_transform__bin_discretizer__strategy': ['uniform','quantile','kmeans'], 
            'pre__bin_transform__bin_discretizer__encode': ['ordinal'],
            'pre__bin_transform__bin_scaler': scaler_list,
            'sel__n_features_to_select': randint(40, 80),
            'sel__step': [2],
            'ovr__n_neighbors': randint(2, 6),
            'clf__bootstrap': [True],
            'clf__ccp_alpha': loguniform(1e-05, 1e-00),                                   # Cost-complexity pruning is an algorithm used to prune a tree to avoid over-fitting
            'clf__criterion': ['gini'],
            'clf__max_depth': randint(60, 90),
            'clf__max_features': loguniform(0.30, 0.70), 
            'clf__min_impurity_decrease': loguniform(1e-10, 1e-06),
            #'clf__max_samples': loguniform(0.05, 0.40),                                  # Only for bootstrap=True. Including this parameter significantly reduced model performance. 
            'clf__min_samples_leaf': loguniform(0.05, 0.25),
            'clf__min_samples_split': loguniform(0.05, 0.25),
            'clf__min_weight_fraction_leaf': loguniform(0.05, 0.25),
            'clf__oob_score': [False],                                                    # Only for bootstrap=True
            'clf__warm_start': [True],
            'clf__n_jobs': [6],
            'clf__random_state': [67],
        }

    elif model_name == 'ExtraTrees':
        params = {
            'pre__cat_transform__cat_encoder__sigma': loguniform(0.001, 0.05),
            'pre__num_transform__num_scaler': scaler_list,
            'pre__bin_transform__bin_discretizer__n_bins': randint(10, 25),
            'pre__bin_transform__bin_discretizer__strategy': ['uniform','quantile','kmeans'], 
            'pre__bin_transform__bin_discretizer__encode': ['ordinal'],
            'pre__bin_transform__bin_scaler': scaler_list,
            'sel__n_features_to_select': randint(35, 85),
            'sel__step': [2],
            'ovr__n_neighbors': randint(2, 6), 
            'clf__bootstrap': [False],
            'clf__ccp_alpha': loguniform(1e-08, 1e-03),                                   # Cost-complexity pruning is an algorithm used to prune a tree to avoid over-fitting
            'clf__criterion': ['entropy'],
            'clf__max_depth': randint(50, 85),
            'clf__max_features': loguniform(0.20, 0.55),
            'clf__max_leaf_nodes': randint(30, 75),
            #'clf__max_samples': loguniform(0.10, 0.50),                                  # Only for bootstrap=True
            'clf__min_impurity_decrease': loguniform(1e-10, 1e-07),
            'clf__min_samples_leaf': loguniform(0.001, 0.10),
            'clf__min_samples_split': loguniform(0.001, 0.10),
            'clf__min_weight_fraction_leaf': loguniform(0.001, 0.10),
            'clf__oob_score': [False],                                                    # Only for bootstrap=True
            'clf__warm_start': [False],
            'clf__n_jobs': [6],
            'clf__random_state': [67],
        }

    elif model_name == 'GradientBoosting':
        params = {
            'pre__cat_transform__cat_encoder__sigma': loguniform(0.001, 0.05),
            'pre__num_transform__num_scaler': scaler_list,
            'pre__bin_transform__bin_discretizer__n_bins': randint(10, 25),
            'pre__bin_transform__bin_discretizer__strategy': ['uniform','quantile','kmeans'], 
            'pre__bin_transform__bin_discretizer__encode': ['ordinal'],
            'pre__bin_transform__bin_scaler': scaler_list,
            'sel__n_features_to_select': randint(30, 70),
            'sel__step': [2],
            'ovr__n_neighbors':randint(2, 6),
            'clf__criterion': ['friedman_mse'],
            'clf__ccp_alpha': loguniform(1e-10, 1e-06),                                   # Cost-complexity pruning is an algorithm used to prune a tree to avoid over-fitting
            'clf__learning_rate': loguniform(1e-02, 1e-00),
            'clf__loss': ['exponential'],
            'clf__max_depth': randint(30, 65),
            'clf__max_features': loguniform(0.20, 0.55), 
            'clf__max_leaf_nodes': randint(50, 80),
            'clf__min_weight_fraction_leaf': loguniform(0.30, 0.50),                      # Must be <= 0.5
            'clf__min_impurity_decrease': loguniform(1e-10, 1e-07),
            'clf__min_samples_leaf': loguniform(0.005, 0.15),
            'clf__min_samples_split': loguniform(0.02, 0.25),
            'clf__n_iter_no_change': [150],
            'clf__tol': loguniform(1e-10, 1e-06),
            'clf__validation_fraction': [0.07],
            'clf__warm_start': [True],
            'clf__subsample': loguniform(0.95, 1.0),
            'clf__random_state': [67],
        }

    elif model_name == 'AdaBoost':
        params = {
            'pre__cat_transform__cat_encoder__sigma': loguniform(0.001, 0.05),
            'pre__num_transform__num_scaler': scaler_list,
            'pre__bin_transform__bin_discretizer__n_bins': randint(10, 25),
            'pre__bin_transform__bin_discretizer__strategy': ['uniform','quantile','kmeans'], 
            'pre__bin_transform__bin_discretizer__encode': ['ordinal'],
            'pre__bin_transform__bin_scaler': scaler_list,
            'sel__n_features_to_select': randint(5, 40),
            'sel__step': [2],
            'ovr__n_neighbors': randint(2, 6), 
            'clf__algorithm': ['SAMME'],
            'clf__learning_rate': loguniform(1e-03, 1e-00),
            'clf__random_state': [67],
        }

    elif model_name == 'XGBoost':
        params = {
            'pre__cat_transform__cat_encoder__sigma': loguniform(0.001, 0.05),
            'pre__num_transform__num_scaler': scaler_list,
            'pre__bin_transform__bin_discretizer__n_bins': randint(10, 25),
            'pre__bin_transform__bin_discretizer__strategy': ['uniform','quantile','kmeans'], 
            'pre__bin_transform__bin_discretizer__encode': ['ordinal'],
            'pre__bin_transform__bin_scaler': scaler_list,
            'sel__n_features_to_select': randint(5, 40),
            'sel__step': [2],
            'ovr__n_neighbors': randint(2, 6), 
            'clf__tree_method': ['hist'],                                                      # 'hist' is faster than 'auto'
            'clf__booster': ['gbtree'],                                                        # 'gbtree' is faster than 'gblinear'
            'clf__eta': [0.3],
            'clf__max_depth': [8],
            'clf__max_bin': [256],
            'clf__grow_policy': ['depthwise','lossguide'],                                     # 'lossguide','depthwise'
            'clf__objective': ['binary:hinge'],
            'clf__eval_metric': ['logloss'],
            'clf__seed': [67],
        }
    
    search = HalvingRandomSearchCV(
        estimator=pipeline,
        param_distributions=params,
        scoring='accuracy',
        factor=halving_parameter,
        resource='clf__n_estimators',
        n_candidates='exhaust',
        min_resources=min_resource,
        max_resources=max_resource,
        aggressive_elimination=False,
        return_train_score=True,
        refit=True,
        cv=n_cross_validation,
        n_jobs=-1,
        random_state=67,
        verbose=1,
        error_score='raise',
    )
    
    start_training_time = time.time()
    search.fit(X_train, y_train)
    
    # Print training and test results
    print('--------------------------')
    print(' ')
    end_training_time = time.time()
    print(f"{model_name} runtime:", round((end_training_time - start_training_time) / 60, 0), 'minutes')
    print(f"{model_name} best training score: {search.best_score_}")
    print(f"{model_name} best test score: {search.score(X_test, y_test)}")
    print(' ')
    print(f"{model_name} parameters: {search.best_params_}")
    print(' ')

    # Print features selected during recursive feature selection (RFE)
    selected_features = zip(search.best_estimator_.named_steps['sel'].ranking_, initial_features, search.best_estimator_.named_steps['sel'].support_)
    selected_features = [x for x in selected_features if x[2] == True]
    selected_features = [x[1] for x in selected_features]
    print(f"{model_name} features: {selected_features}")
    print(' ')
    
    # Print classification report and confusion matrix
    print(classification_report(y_test, search.predict(X_test), target_names=['run','pass'], digits=4))
    cm = confusion_matrix(y_test, search.predict(X_test))
    ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['run','pass']).plot(cmap=plt.cm.Blues) 
    plt.show()
    
    print(' ')
    print('-------------------------------------------------------------------------------------------------------------------------------')
    print("\n")

In [ ]:
# Calculate workbook processing time in hours
end_time = time.time()
total_time = end_time - start_time

print('Total HalvingRandomSearchCV runtime', round(total_time / 3600, 2), 'hours')